In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import linear_model,ensemble,tree
from sklearn.model_selection import train_test_split

def file_reader(file_location):
    if(file_location.endswith('csv')):
        return pd.read_csv( file_location , engine = 'python', index_col=0)
    elif (file_location.endswith('tsv')):
        return pd.read_csv( file_location , engine = 'python' ,sep = '\t')
    
def year_str2float(year_quarter):
    if ( year_quarter.endswith('Q1') ):
        return int(year_quarter[:-2]) + 0.25
    elif( year_quarter.endswith('Q2') ):
        return int(year_quarter[:-2]) + 0.50
    elif( year_quarter.endswith('Q3') ):
        return int(year_quarter[:-2]) + 0.75
    elif( year_quarter.endswith('Q4') ):
        return int(year_quarter[:-2]) + 1.00
    
def get_data(country):
    x        = []
    timeline = [ year_str2float(val) for val in dfx['above65.csv'].index ]
    x.append( timeline )
    for str in strlist:
        x.append( dfx[str][country] )
    y        = dfy[country]
    x        = np.array(x).T
    return x,y

def model_fit(x,y, n):
    from sklearn import linear_model
    #model = linear_model.LinearRegression()
    model    = ensemble.RandomForestRegressor(n_estimators=n, criterion ='mse',random_state = 0)
    #model    = tree.DecisionTreeRegressor(random_state=0)
    model.fit(x, y)
    print('R2 Train Score :' ,model.score(x, y))
    return model

In [2]:
strlist = [
    'above65.csv',
    #'area.csv',
    #'literacy_rate.csv',
    'corona_cases.csv',
    'mobile_users.csv',
    'total_pop.csv',
    'unemployed.csv'
    
      ]

In [3]:
dfx = {}
for str in strlist:
    dfx[str] = file_reader('dataset/'+str)
dfy = file_reader('dataset/gdp_change.csv')
countries = file_reader('dataset/Downloaded Dataset/countries.csv')

In [6]:
xtotal , ytotal = [] ,[]
for country in countries['Country']:
    x,y = get_data(country)
    for t in x[:-1]:
        xtotal .append(t)
    for t in y:
        ytotal .append(t)

In [7]:
xtrain, xtest, ytrain, ytest = train_test_split( xtotal, ytotal, test_size=0.3, random_state=0)


In [9]:
#for n in range(1 , 10, 1):
    model                        = model_fit(xtrain,ytrain, 4)
    model.score(xtest, ytest)
    q3preds = []
    for country in countries['Country']:
        x,y   = get_data(country)
        preds = model.predict(x[-1:])
        q3preds.append( preds[0] )

    predicted = dfy.T
    predicted['2020Q3_Predicted'] = q3preds
    #predicted['2020Q4'] = q4preds
    print('\n', predicted[['2020Q3_Predicted']],'\n' )

R2 Train Score : 0.9620280775204049

 Year/Quarter    2020Q3_Predicted
Australia              -6.366524
Brazil                 -6.400000
Canada                 -7.639337
United States          -6.000000
United Kingdom         -7.250483
France                -13.755489
Spain                 -13.104801
India                 -14.956022
Singapore             -10.100000
Thailand               -7.908243
China                  -1.498642
Greece                -12.843154 

